In [1]:
import os
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition'

In [11]:
#Entity 

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    input_shape: int
    NumOfClasses: int
    Conv1D_1: int
    Dropout1: float
    Conv1D_2: int
    Dropout2: float
    pool_size: int
    Dense_layer: int

In [12]:
from Sign_Language.constants import *
from Sign_Language.utils.common import create_directories, read_yaml

In [19]:
# Configuration Manager

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        params = self.params.baseModelParams


        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
           
            NumOfClasses = params.NumOfClasses,
            input_shape = params.input_shape,
            Conv1D_1 = params.Conv1D_1,
            Dropout1 = params.Dropout1,
            Conv1D_2 = params.Conv1D_2,
            Dropout2 = params.Dropout2,
            pool_size = params.pool_size,
            Dense_layer = params.Dense_layer
        )

        return prepare_base_model_config


In [15]:
from Sign_Language import logger
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense

In [24]:
# #Component

class PrepareModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def create_model(self):
        self.model = Sequential()

        # ---------------------------------- CNN Model ---------------------------------- #
        # Convolutional Layer 1
        self.model.add(Conv1D(filters=self.config.Conv1D_1, kernel_size=3, activation='relu', input_shape=(self.config.input_shape, 1)))
        self.model.add(Dropout(self.config.Dropout1))

        # Convolutional Layer 2
        self.model.add(Conv1D(filters=self.config.Conv1D_2, kernel_size=3, activation='relu'))
        self.model.add(Dropout(self.config.Dropout2))

        # Max Pooling Layer
        self.model.add(MaxPooling1D(pool_size=2))

        # Flatten Layer
        self.model.add(Flatten())

        # Fully Connected Layer 1
        self.model.add(Dense(self.config.Dense_layer, activation='relu'))

        # Fully Connected Layer 2
        self.model.add(Dense(28, activation='softmax'))
        # ---------------------------------------------------------------------------------- #

        # Print model summary
        self.model.summary()
        # Compile model
        # self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.save_model(self.config.base_model_path, self.model)
        logger.info("Created the Base Model:")


    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



In [25]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.prepare_base_model_config()
    prepare_base_model = PrepareModel(config=prepare_base_model_config)
    prepare_base_model.create_model()
except Exception as e:
    raise e

[2024-01-24 20:39:58,543: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-24 20:39:58,549: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-24 20:39:58,553: INFO: common: created directory at: artifacts]
[2024-01-24 20:39:58,556: INFO: common: created directory at: artifacts/prepare_base_model]


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 40, 64)            256       
                                                                 
 dropout_4 (Dropout)         (None, 40, 64)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 38, 128)           24704     
                                                                 
 dropout_5 (Dropout)         (None, 38, 128)           0         
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 19, 128)           0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2432)              0         
                                                      